### Логистическая регрессия

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2,  1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [4]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

#### Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [5]:
# Добавим небольшое значение к аргументу в np.log
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred + 1e-9) + (1.0 - y) * np.log(1.0 - y_pred + 1e-9))
    err = np.sum(err)
    return err

In [6]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

#### Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [7]:
# Логистическая регрессия
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(f'Итерация {i:5}, {W}, Log Loss: {err}')
    return W

In [8]:
W = eval_model(X_st, y, iterations=10000, alpha=3.2)

Итерация  1000, [-24.66166323  -2.58625863  -6.4883687   20.51862732], Log Loss: 0.14515608431836463
Итерация  2000, [-30.9436166   -3.17482029  -8.28595043  25.51449064], Log Loss: 0.12175660055634288
Итерация  3000, [-35.62919029  -3.59578876  -9.60752257  29.29589069], Log Loss: 0.10976714562691066
Итерация  4000, [-39.46740432  -3.93563493 -10.69080562  32.37822401], Log Loss: 0.10176998098456422
Итерация  5000, [-42.77500088  -4.22300941 -11.62764808  35.02045176], Log Loss: 0.09585970833393595
Итерация  6000, [-45.72133652  -4.4734476  -12.46641145  37.36113568], Log Loss: 0.09119036206038213
Итерация  7000, [-48.40697954  -4.69628808 -13.23547143  39.4826709 ], Log Loss: 0.08732668143753532
Итерация  8000, [-50.8962043   -4.89759487 -13.95275545  41.43786763], Log Loss: 0.08402054876576308
Итерация  9000, [-53.23234596  -5.0815578  -14.63019366  43.26251651], Log Loss: 0.08111951759436889
Итерация 10000, [-55.44580972  -5.25122724 -15.27602267  44.9819372 ], Log Loss: 0.07852445

#### Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1   
(на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba)

In [9]:
W

array([-55.44580972,  -5.25122724, -15.27602267,  44.9819372 ])

In [10]:
def calc_pred_proba(w, x):
    y_pred_proba = 1/(1+np.exp(-np.dot(w,x)))
    return y_pred_proba

In [11]:
calc_pred_proba(W, X_st)

array([3.20632749e-01, 8.65276587e-04, 1.00000000e+00, 1.52546073e-11,
       9.78121910e-01, 7.02846565e-03, 1.00000000e+00, 4.94947027e-03,
       6.95151970e-01, 1.00000000e+00])

In [12]:
def calc_pred_proba2(w, x):
    y_pred_proba = sigmoid(np.dot(w,x))
    return y_pred_proba

In [13]:
calc_pred_proba2(W, X_st)

array([3.20632749e-01, 8.65276587e-04, 1.00000000e+00, 1.52546073e-11,
       9.78121910e-01, 7.02846565e-03, 1.00000000e+00, 4.94947027e-03,
       6.95151970e-01, 1.00000000e+00])

#### Создайте функцию calc_pred, возвращающую предсказанный класс   
(на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [14]:
def calc_pred(w, x):
    return np.array([1 if x > 0.5 else 0 for x in calc_pred_proba(w, x)])

In [15]:
y_pred = calc_pred(W, X_st)
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

#### Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [16]:
# Доля правильных ответов
accuracy = 100.0 - np.mean(np.abs(y_pred - y)*100.0)
print(f'Accuracy: {accuracy:.2f}%')

Accuracy: 100.00%


In [17]:
# Матрица ошибок
def confusion_matrix(y, y_pred):
    k = len(np.unique(y)) 
    result = np.zeros((k, k))
    for fact, pred in zip(y, y_pred):
        if fact == pred:
            if fact:
                result[0,0] += 1
            else:
                result[1,1] += 1
        else:
            if fact:
                result[1,0] += 1
            else:
                result[0,1] += 1
    return result

In [18]:
cm = confusion_matrix(y, y_pred)
print('Матрица ошибок:\n')
print('\tФакт+\tФакт-')
print(f'1\t{cm[0][0]}\t{cm[0][1]}')
print(f'0\t{cm[1][0]}\t{cm[1][1]}')

Матрица ошибок:

	Факт+	Факт-
1	5.0	0.0
0	0.0	5.0


In [19]:
precision = cm[0][0] / (cm[0][0] + cm[0][1])
print(f'Точность: {precision}')

Точность: 1.0


In [20]:
recall = cm[0][0] / (cm[0][0] + cm[1][0])
print(f'Полнота: {recall}')

Полнота: 1.0


In [21]:
f1 = 2 * precision * recall / (precision + recall)
print(f'F1 score: {f1}')

F1 score: 1.0


#### Могла ли модель переобучиться? Почему?

Да, могла переобучиться, так как в наборе данных мало значений.